%md

# EEET2574 | Assignment 2: Data Pipeline for Dutch Energy

Name: Pham Xuan Dat 

Student ID: s3927188

# Task 1: MongoDB

## Q1: How many collections do you have? Why?

Although both `Gas` and `Electricity` dataset contains the same columns name, the values in which exert inherently different patterns (which are discussed in model training modules). Therefore, having a separate collection for each category would prove more beneficial for future scalability and visualization. 

Apart from the original columns, I have added another 2 columns `company` and  `year`. The former column marks the author of the energy records and the latter presents the document's date. Overall, the two collections' schema is shown in below figures.

![alt text](https://i.imgur.com/9J3pj1i.png)

![alt text](https://i.imgur.com/cqGjql5.png)

# Task 2: 

## Q2-A: What are the chosen data cleaning steps? Why?

Our data cleaning process involve two main steps: handling missing data and outliers.

- **Handling Missing Data**: We define a threshold for which a column or row will be drop for a certain portion of missing values. For instance, we would drop rows or columns if 50% of its values are missing.

- **Handling Outliers**: First, we adopt Interquartile Range (IQR) in detecting the outliers. We could either remove data points that fall outside the quarter ranges, or leave it for later transformation (which is discussed in Q2-B). 

## Q2-B: What are the chosen data transformation steps? Why?

Our data cleaning process involve three main steps: Feature Engineering, Categorical Encoding and Normalization. 

- **Categorical Encoding**: 

This is an essential step in presenting categorical data in numeric values. Several options have been carefully tested. For instance, `OneHotEncoder` convert distinct values into binary vectors, which had drastically surged the data size when one column contains over 90,000 distinct values. We decided to choose `HashingEncoder`, which is more limited in uniqueness in exchange for reasonable size. 

- **Feature Engineering**: 

This is to helps mitigate the size issue mentioned above. For instance, we could merge two columns `zipcode_from` and `zipcode_to`, both of which contains over 90,000 unique values. This is rather tradeoff between accuracy and speed, in which we de-specify data in exchange for less processing workload. 

- **Normalization**: 

This is essential for ensuring that features are on a similar scale, from hundred of thousands down to a fixed range. Due to a large proportion of outliers detected in the dataset, we utilized `RobustScaler` for transforming the outliers rather than complete removal, which may affects the overall correlation between the data points.  

# Task 3: Model training and tracking with data pipeline and MLflow

Since variety of models can be experimented during development, we have develop a generic interface, aiming to shorten the workflow from initiating the pipelines and data to final model evaluation.

A whole training process commences with preparing the dataset with a model instance (e.g., ElasticNet) with desired parameters for tuning. Next, the main function start hypertuning with `GridSearchCV`, and loggin processed model with `MlFlow`. The training concludes with records of generated models, along with their evaluation metrics.  

![alt text](https://i.imgur.com/PWCpKvr.png)

Our models comprised two tree-based methods and a regularized method

- Tree-based methods: **XGBoost, Random Forest** with evaluation metrics focused on `RMSE` and `R2`

- Regularized method: **Elastic Net** with evaluation metrics focused on `MAE` and `R2`

## Q3-A,B: What is/are your final model(s) based on the evaluation metrics? Did you build one model for both electricity and gas or separate models? Why?

Since GridsearchCV inherently generate signifcant amount of models every run, we only log models that pass certain thresholds, and finally sort them by desired metrics.

```python
_params_grid = { 
    'n_estimators':[38, 44, 50],
    'max_depth': [12],      # Max depth of trees
    ...
}
_estimator = CustomEstimatorRF(random_state=42, n_jobs=1)
_model_pipeline = make_pipeline(clean_pipeline_rf, process_pipeline_rf)
_metrics = ('neg_mean_squared_error', 'neg_rmse') # find model with lowest RMSE

start_train((df_train, df_test), _params_grid,
            _estimator, _model_pipeline, 
            _metrics, False)
```

The results after applying 3 models for both Gas and Electricity are captured by MlFLow

- Electricity

![mlflow](https://i.imgur.com/PLeYbH9.png)

![mlflow](https://i.imgur.com/w9jV51o.png)

```python
(ElasticNet)
> Best Model
  Params: {'alpha': 0.1, 'l1_ratio': 0.01, 'max_iter': 1000}
  RMSE (train): 2754.855349869532
  MAE (train): 1454.769370137224
  R2 (train): 0.2951528574811394
  RMSE: 2647.080508350842
  MAE: 1472.0398544352042
  R2: 0.3275408730101492

(XGBoost)
> Best Model
  Params: {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 100, 'n_estimators': 50, 'reg_alpha': 67.5, 'reg_lambda': 67.5, 'subsample': 0.5}
  RMSE (train): 1789.1889366152327
  MAE (train): 826.3288306144177
  R2 (train): 0.7026898395520302
  RMSE: 1983.7239550179977
  MAE: 1033.4624518183257
  R2: 0.6223460184435804

(Random Forest)
> Best Model
  Params: {'max_depth': 12, 'max_features': 'log2', 'min_samples_leaf': 18, 'min_samples_split': 25, 'n_estimators': 38}
  RMSE (train): 2670.234796574073
  MAE (train): 1404.2687322200227
  R2 (train): 0.3377892216407463
  RMSE: 2547.9730768107843
  MAE: 1408.0918260853866
  R2: 0.3769523510198989
```

- Gas

![mlflow](https://i.imgur.com/qM92ynF.png)

![mlflow](https://i.imgur.com/NKj81ih.png)

```python
(ElasticNet)
> Best Model
  Params: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': 18, 'max_features': 'log2', 'max_leaf_nodes': 100, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 18, 'min_samples_split': 25, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 38, 'n_jobs': 1, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}
  RMSE (train): 758.8313459628483
  MAE (train): 470.4356213305831
  R2 (train): 0.37744838730921704
  RMSE: 928.8693616777718
  MAE: 614.0884675362854
  R2: 0.21177241531996482> Best Model
  Params: {'alpha': 0.1, 'copy_X': True, 'fit_intercept': True, 'l1_ratio': 0.01, 'max_iter': 1000, 'positive': False, 'precompute': False, 'random_state': 42, 'selection': 'cyclic', 'tol': 0.0001, 'warm_start': False}
  RMSE (train): 716.2469047322095
  MAE (train): 458.2401509250702
  R2 (train): 0.44536106433385814
  RMSE: 884.256992688362
  MAE: 497.5477595307483
  R2: 0.2856692284123472

(XGBoost)
> Best Model
  Params: {'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 1, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.1, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 75, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 25, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': 42, 'reg_alpha': 67.5, 'reg_lambda': 67.5, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': 0.5, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}
  RMSE (train): 585.1212103734053
  MAE (train): 355.8841670672757
  R2 (train): 0.6298510513354676
  RMSE: 771.4878764435983
  MAE: 460.4134358265588
  R2: 0.45624837917637484

(RandomForest)
> Best Model
  Params: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': 18, 'max_features': 'log2', 'max_leaf_nodes': 100, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 18, 'min_samples_split': 25, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 38, 'n_jobs': 1, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}
  RMSE (train): 758.8313459628483
  MAE (train): 470.4356213305831
  R2 (train): 0.37744838730921704
  RMSE: 928.8693616777718
  MAE: 614.0884675362854
  R2: 0.21177241531996482
```

From the metrics illustrated above, `XGBoost` is our final choice for both Gas and Electricity category as it yielded lowest `RMSE`, `MAE` with highest `R2`. 

Nevertheless, I think the model would work best if fed with two separate dataset. The reason is that although both have same data columns, our analysis showed that they are influenced by different factors and exhibit different usage patterns. For instance, `type_of_connection` for electricity involve the number of Fuses and Ampere, whereas `annual_consume_lowtarif_perc` for gas is always `0`. By developing separate models, the unique characteristics and predictors relevant to each energy type can be captured more accurately, leading to better performance and more precise forecasts.

## Q3-C: Should we build a separate model for each company or not? Why?

The first highlight in our exploratory analysis and visualization showed that samples across 3 companies were unevenly distributed, with `Stedin` taking over 90%. Given such skewed dataset, having a separate model for each company would not only prevent prediction bias, but also gain further insights in company-specific nature (e.g., consumption behavior, most-focused areas). 

# Task 4: Visualization

[MongoDB Charts Dashboard](https://charts.mongodb.com/charts-bigd-test1-begvykr/public/dashboards/676ce810-55a5-476a-8259-eb614b7fe1d6#)